pre process

In [1]:
import torch

if torch.backends.mps.is_available():
    print("MPS is available!")
else:
    print("MPS is not available. Check your PyTorch installation.")

MPS is available!


In [2]:
from transformers import AutoTokenizer
classificaiton = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(classificaiton)
tokens = tokenizer("Hello World, this is akshay")
print(tokens)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'input_ids': [101, 7592, 2088, 1010, 2023, 2003, 17712, 7377, 2100, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [3]:
batch_data = [
    "Hi my name is akshay",
    "I am working at axtr labs",
    "i like my work"
]
token_batch = tokenizer(batch_data,padding=True,return_tensors="pt")
print(token_batch)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

{'input_ids': tensor([[  101,  7632,  2026,  2171,  2003, 17712,  7377,  2100,   102],
        [  101,  1045,  2572,  2551,  2012, 22260, 16344, 13625,   102],
        [  101,  1045,  2066,  2026,  2147,   102,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0]])}


fiine tun|


In [4]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [6]:
dataset["train"]

Dataset({
    features: ['label', 'text'],
    num_rows: 650000
})

In [7]:
dataset["train"][1]


{'label': 1,
 'text': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."}

In [8]:
dataset["train"].features

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars'], id=None),
 'text': Value(dtype='string', id=None)}

In [9]:
def tokenizerFunction(example):
    return tokenizer(example["text"],padding="max_length",max_length=128,truncation=True)
tokenized_dataset = dataset.map(tokenizerFunction,batched=True
                                )

In [10]:
small_tokenized_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(100))

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(classificaiton, num_labels= 5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
!pip install scikit-learn

  Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl (11.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [14]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

In [15]:
from transformers import TrainingArguments,Trainer
args = TrainingArguments(
    output_dir="/Users/akshay/Documents/Work/fineTuning",
    eval_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,  # Log every 10 steps
    report_to="wandb",  # Disable Wandb
)

In [17]:
def compute_metrices(eval_pred):
    logits, labels = eval_pred
    predcitions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predcitions,references=labels)
import torch

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(device)
model.to(device)

mps


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [19]:
trainer = Trainer(model=model,compute_metrics=compute_metrices,args=args,train_dataset=small_tokenized_dataset,eval_dataset=small_eval_dataset)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: akshay-sk1906 (akshay-sk1906-psg-college-of-technology). Use `wandb login --relogin` to force relogin


                                              
 33%|███▎      | 7/21 [01:57<04:02, 17.31s/it] 

{'eval_loss': 1.6108647584915161, 'eval_accuracy': 0.19, 'eval_runtime': 30.5803, 'eval_samples_per_second': 3.27, 'eval_steps_per_second': 0.425, 'epoch': 1.0}


In [30]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Use the path where your model was saved during training
model_path = "/Users/akshay/Documents/Work/fineTuning/checkpoint-21"  # Adjust this path
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [47]:
import torch
def prediction(text):
    inputs = tokenizer(text , padding=True , truncation=True , return_tensors="pt",max_length=128)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    predictions = torch.nn.functional.softmax(outputs.logits , dim =-1)
    predicted_class = torch.argmax(predictions , dim=1) 
    sequence_label = ["1 Star","2 Star","3 Star","4 Star","5 Star"]
    return {
        "prediction" : sequence_label[predicted_class],
        "confidence_level" : predictions[0].tolist()
    }
    


In [49]:
# Example usage
test_texts = [
    "This product is amazing! I love it completely.",
    "Terrible experience, would not recommend.",
    "It's okay, nothing special."
]
for inputs in test_texts:
    outputs = prediction(inputs)
    print(inputs)
    print(f"Precicted Score: { outputs["prediction"] }")
    print(f"Confindence : { outputs["confidence_level"]}")


This product is amazing! I love it completely.
Precicted Score: 5 Star
Confindence : [0.06481587886810303, 0.2333972454071045, 0.09326429665088654, 0.20702354609966278, 0.4014990031719208]
Terrible experience, would not recommend.
Precicted Score: 2 Star
Confindence : [0.08969159424304962, 0.401839017868042, 0.10757553577423096, 0.21330028772354126, 0.18759362399578094]
It's okay, nothing special.
Precicted Score: 2 Star
Confindence : [0.12948758900165558, 0.3454606831073761, 0.12496819347143173, 0.14739955961704254, 0.2526840567588806]
